# ☀️ Solar Potential LSTM — Training Notebook

Train an LSTM model to predict **solar energy potential** for a building rooftop,
based on a 24-step time-series of environmental and building conditions.

**Inputs (per time-step):**
- `peak_sun_hours` — daily peak sun hours (3–7 hrs)
- `shadow_coverage` — fraction of roof under shadow (0–0.6)
- `temperature_c` — ambient temperature (°C)
- `cloud_cover` — cloud cover fraction (0–1)
- `roof_area_m2` — total roof area (m²)
- `tariff_inr` — electricity tariff (₹/kWh)

**Outputs:**
- `system_size_kw` — estimated solar system capacity
- `energy_month_kwh` — monthly energy generation
- `savings_month_inr` — monthly financial savings (₹)
- `effective_sun_hours` — sun hours after shadow loss
- `usable_roof_area_m2` — usable area for panels

Saves:
- `models/solar_lstm.pth`
- `models/solar_scaler.json`

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import json
import os
import matplotlib.pyplot as plt

print(f"PyTorch {torch.__version__}")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# ======================== CONFIG ========================
SEQ_LEN      = 24       # 24 time-steps (hourly readings for a day)
INPUT_DIM    = 6        # sun_hours, shadow, temp, cloud, roof_area, tariff
OUTPUT_DIM   = 5        # system_kw, energy_month, savings_month, sun_hrs, usable_area
HIDDEN_DIM   = 64
NUM_LAYERS   = 2
DROPOUT      = 0.2
EPOCHS       = 100
BATCH_SIZE   = 32
LR           = 0.001
NUM_SAMPLES  = 5000

# Constants from calculations.ts
USABILITY_FACTOR    = 0.7    # 70% of roof usable
SQM_PER_KW          = 10.0   # 1 kW needs ~10 m²
PERFORMANCE_RATIO   = 0.8    # 80% system efficiency

In [ ]:
# ===================== DATA GENERATION =====================
def generate_solar_data(num_samples: int):
    """
    Generate synthetic time-series data for solar potential prediction.
    Formula mirrors calculations.ts -> calculateSolarPotential().
    
    Time-series captures hourly variations in sun intensity, temperature,
    and cloud cover to create realistic diurnal patterns.
    """
    np.random.seed(42)
    X, y = [], []

    for _ in range(num_samples):
        # Base conditions (randomised per sample)
        base_sun_hours    = np.random.uniform(3.0, 7.0)       # peak sun hrs
        base_shadow       = np.random.uniform(0.05, 0.55)     # shadow coverage
        base_temp         = np.random.uniform(20, 45)          # °C
        base_cloud        = np.random.uniform(0.0, 0.7)       # cloud cover
        roof_area         = np.random.uniform(50, 600)         # m² roof area
        tariff            = np.random.uniform(5.0, 12.0)       # ₹/kWh

        seq = []
        for t in range(SEQ_LEN):
            # Diurnal variation: sun peaks at noon (t=12), clouds vary
            hour_factor = np.clip(np.sin((t - 6) * np.pi / 12), 0, 1)  # 0 at 6am/6pm, 1 at noon

            sun_hours = base_sun_hours * (0.8 + 0.4 * hour_factor) + np.random.normal(0, 0.2)
            sun_hours = np.clip(sun_hours, 1.0, 8.0)

            shadow = base_shadow + np.random.normal(0, 0.03)
            shadow = np.clip(shadow, 0.0, 0.8)

            temp = base_temp + np.random.normal(0, 2) + np.sin(t * np.pi / 12) * 5
            cloud = base_cloud + np.random.normal(0, 0.05) * (1 - hour_factor * 0.3)
            cloud = np.clip(cloud, 0.0, 1.0)

            seq.append([sun_hours, shadow, temp, cloud, roof_area, tariff])

        X.append(seq)

        # ---------- Target: mirrors calculateSolarPotential() ----------
        # Use the final time-step shadow + average sun hours in the sequence
        final_shadow = seq[-1][1]
        avg_sun      = np.mean([s[0] for s in seq])
        final_cloud  = np.mean([s[3] for s in seq])

        # Cloud cover reduces effective sun hours
        effective_peak_sun = avg_sun * (1.0 - final_cloud * 0.5)

        # Step 1: Usable roof area
        usable_roof_area = roof_area * USABILITY_FACTOR * (1 - final_shadow)

        # Step 2: System size (1 kW per 10 m²)
        system_size_kw = usable_roof_area / SQM_PER_KW

        # Step 3: Energy generation
        # Temperature coefficient: panels lose ~0.4% efficiency per °C above 25°C
        avg_temp = np.mean([s[2] for s in seq])
        temp_coeff = 1.0 - max(0, (avg_temp - 25)) * 0.004
        temp_coeff = np.clip(temp_coeff, 0.8, 1.0)

        energy_year_kwh  = system_size_kw * 365 * effective_peak_sun * PERFORMANCE_RATIO * temp_coeff
        energy_month_kwh = energy_year_kwh / 12

        # Step 4: Savings
        savings_month_inr = (energy_year_kwh * tariff) / 12

        # Effective sun hours
        effective_sun_hours = effective_peak_sun * (1 - final_shadow)

        y.append([
            system_size_kw,
            energy_month_kwh,
            savings_month_inr,
            effective_sun_hours,
            usable_roof_area,
        ])

    return np.array(X, dtype=np.float32), np.array(y, dtype=np.float32)


X, y = generate_solar_data(NUM_SAMPLES)
print(f"X shape: {X.shape}  |  y shape: {y.shape}")
print(f"Sample input (last step): {X[0][-1]}")
print(f"Sample target: {y[0]}")

In [ ]:
# =================== NORMALISATION ===================
x_flat = X.reshape(-1, INPUT_DIM)
x_min  = x_flat.min(axis=0)
x_max  = x_flat.max(axis=0)
y_min  = y.min(axis=0)
y_max  = y.max(axis=0)

X_norm = (X - x_min) / (x_max - x_min + 1e-8)
y_norm = (y - y_min) / (y_max - y_min + 1e-8)

# Train / Validation split (80 / 20)
split = int(0.8 * NUM_SAMPLES)
X_train, X_val = X_norm[:split], X_norm[split:]
y_train, y_val = y_norm[:split], y_norm[split:]

train_ds = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
val_ds   = TensorDataset(torch.FloatTensor(X_val),   torch.FloatTensor(y_val))

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE)

print(f"Train: {len(train_ds)} | Val: {len(val_ds)}")
print(f"\nFeature ranges:")
feature_names = ["sun_hours", "shadow", "temp", "cloud", "roof_area", "tariff"]
for i, name in enumerate(feature_names):
    print(f"  {name:>12}: [{x_min[i]:.2f}, {x_max[i]:.2f}]")

In [ ]:
# ==================== LSTM MODEL ====================
class SolarLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout=0.2):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(
            input_dim, hidden_dim, num_layers,
            batch_first=True, dropout=dropout
        )
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim, 32),
            nn.ReLU(),
            nn.Linear(32, output_dim),
        )

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        return self.fc(out[:, -1, :])


model     = SolarLSTM(INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM, NUM_LAYERS, DROPOUT).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10, factor=0.5)

total_params = sum(p.numel() for p in model.parameters())
print(f"SolarLSTM  |  Parameters: {total_params:,}")
print(model)

In [ ]:
# ==================== TRAINING LOOP ====================
train_losses = []
val_losses   = []

for epoch in range(EPOCHS):
    # --- train ---
    model.train()
    epoch_loss = 0.0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        pred = model(xb)
        loss = criterion(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    avg_train = epoch_loss / len(train_loader)
    train_losses.append(avg_train)

    # --- validate ---
    model.eval()
    v_loss = 0.0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            v_loss += criterion(model(xb), yb).item()
    avg_val = v_loss / len(val_loader)
    val_losses.append(avg_val)
    scheduler.step(avg_val)

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1:3d}/{EPOCHS}  Train: {avg_train:.6f}  Val: {avg_val:.6f}")

print("\n✅ Training complete.")

In [ ]:
# ==================== LOSS CURVES ====================
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses,   label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.title("Solar LSTM — Training Progress")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# ==================== SAVE MODEL & SCALER ====================
os.makedirs("models", exist_ok=True)

torch.save(model.state_dict(), "models/solar_lstm.pth")

scaler_data = {
    "x_min": x_min.tolist(),
    "x_max": x_max.tolist(),
    "y_min": y_min.tolist(),
    "y_max": y_max.tolist(),
}
with open("models/solar_scaler.json", "w") as f:
    json.dump(scaler_data, f, indent=2)

print("Saved  models/solar_lstm.pth")
print("Saved  models/solar_scaler.json")
print(f"Model size: {os.path.getsize('models/solar_lstm.pth') / 1024:.1f} KB")

In [ ]:
# ==================== QUICK TEST ====================
model.eval()
test_x = torch.FloatTensor(X_norm[:1]).to(device)

with torch.no_grad():
    pred_norm = model(test_x).cpu().numpy()[0]

pred_actual = pred_norm * (y_max - y_min) + y_min
actual      = y[0]

labels = ["system_size_kw", "energy_month_kwh", "savings_month_inr", "effective_sun_hours", "usable_roof_area_m2"]

print("=" * 60)
print(f"{'Metric':<24} {'Predicted':>14} {'Actual':>14}")
print("-" * 60)
for lbl, p, a in zip(labels, pred_actual, actual):
    print(f"{lbl:<24} {p:>14.2f} {a:>14.2f}")
print("=" * 60)

# JSON output example
result_json = {
    "type": "solar",
    "predictions": {
        lbl: round(float(p), 2) for lbl, p in zip(labels, pred_actual)
    },
}
print("\nJSON output:")
print(json.dumps(result_json, indent=2))